# EOFs package El Nino example (xarray)

Compute and plot the leading EOF of sea surface temperature in the
central and northern Pacific during winter time.

The spatial pattern of this EOF is the canonical El Nino pattern, and
the associated time series shows large peaks and troughs for well-known
El Nino and La Nina events.

Additional requirements for this example:

    * xarray (http://xarray.pydata.org)
    * matplotlib (http://matplotlib.org/)
    * cartopy (http://scitools.org.uk/cartopy/)

This notebook is based on code in

    https://ajdawson.github.io/eofs/latest/examples/elnino_xarray.html

Change history:

MGH 2019-04-15
    - Written based on the Web page.

MGH 2019-07-01
    - The filled contour plot now uses the cmocean balance colour map.

In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import cmocean
import numpy as np
import xarray as xr

from eofs.xarray import Eof
from eofs.examples import example_data_path

Read SST anomalies using the xarray module. The file contains November-March
averages of SST anomaly in the central and northern Pacific.

In [2]:
filename = example_data_path('sst_ndjfm_anom.nc')
sst = xr.open_dataset(filename)['sst']

Create an EOF solver to do the EOF analysis. Square-root of cosine of
latitude weights are applied before the computation of EOFs.

In [3]:
coslat = np.cos(np.deg2rad(sst.coords['latitude'].values))
wgts = np.sqrt(coslat)[..., np.newaxis]
solver = Eof(sst, weights=wgts)

Retrieve the leading EOF, expressed as the correlation between the leading
PC time series and the input SST anomalies at each grid point, and the
leading PC time series itself.

In [4]:
eof1 = solver.eofsAsCorrelation(neofs=1)
pc1 = solver.pcs(npcs=1, pcscaling=1)

Plot the leading EOF expressed as correlation in the Pacific domain.

In [5]:
clevs = np.linspace(-1, 1, 11)
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=190))
fill = eof1[0].plot.contourf(ax=ax, levels=clevs, cmap=cmocean.cm.balance,
                             add_colorbar=False, transform=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, facecolor='w', edgecolor='k')
cb = plt.colorbar(fill, orientation='horizontal')
cb.set_label('correlation coefficient', fontsize=12)
ax.set_title('EOF1 expressed as correlation', fontsize=16)

Plot the leading PC time series

In [6]:
plt.figure()
pc1[:, 0].plot(color='b', linewidth=2)
ax = plt.gca()
ax.axhline(0, color='k')
ax.set_ylim(-3, 3)
ax.set_xlabel('Year')
ax.set_ylabel('Normalized Units')
ax.set_title('PC1 Time Series', fontsize=16)

plt.show()